In [87]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import torch
import json
import requests
from bs4 import BeautifulSoup
import sys


ProjectPath = "/home/liranc6/con_graph"
sys.path.append(ProjectPath)  # Add the parent directory to the sys.path

In [88]:
! gpustat

rambo6                      Sun Apr 21 11:12:01 2024  525.147.05
[0] NVIDIA GeForce RTX 3090 | 25°C,   0 % |     1 / 24576 MB |
[1] NVIDIA GeForce RTX 3090 | 87°C,  99 % |  9829 / 24576 MB | shadi-omari(9826M)
[2] NVIDIA GeForce RTX 3090 | 26°C,   0 % |     1 / 24576 MB |
[3] NVIDIA GeForce RTX 3090 | 23°C,   0 % |     1 / 24576 MB |
[4] NVIDIA GeForce RTX 3090 | 25°C,   0 % |     1 / 24576 MB |
[5] NVIDIA GeForce RTX 3090 | 24°C,   0 % |     1 / 24576 MB |
[6] NVIDIA GeForce RTX 3090 | 25°C,   0 % |     1 / 24576 MB |
[7] NVIDIA GeForce RTX 3090 | 24°C,   0 % |     1 / 24576 MB |


In [89]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

! gpustat

rambo6                      Sun Apr 21 11:12:04 2024  525.147.05
[0] NVIDIA GeForce RTX 3090 | 25°C,   0 % |     1 / 24576 MB |
[1] NVIDIA GeForce RTX 3090 | 87°C,  99 % |  9829 / 24576 MB | shadi-omari(9826M)
[2] NVIDIA GeForce RTX 3090 | 26°C,   0 % |     1 / 24576 MB |
[3] NVIDIA GeForce RTX 3090 | 23°C,   0 % |     1 / 24576 MB |
[4] NVIDIA GeForce RTX 3090 | 25°C,   0 % |     1 / 24576 MB |
[5] NVIDIA GeForce RTX 3090 | 24°C,   0 % |     1 / 24576 MB |
[6] NVIDIA GeForce RTX 3090 | 25°C,   0 % |     1 / 24576 MB |
[7] NVIDIA GeForce RTX 3090 | 24°C,   0 % |     1 / 24576 MB |


In [90]:
data_path = "/mnt/qnap/liranc6/data/con_graph/dataset/arxiv-metadata-oai-snapshot.json"
subset_metadata_path = "/mnt/qnap/liranc6/data/con_graph/dataset/subset_metadata.json"

In [91]:
def get_metadata(data_path):
    with open(data_path, 'r') as f:
        for line in f:
            metadata = json.loads(line)
            yield metadata

In [92]:
metadata = get_metadata(data_path)
for paper in metadata:
    for k, v in paper.items():
        print(f'{k}: {v}')
    break

id: 0704.0001
submitter: Pavel Nadolsky
authors: C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan
title: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies
comments: 37 pages, 15 figures; published version
journal-ref: Phys.Rev.D76:013009,2007
doi: 10.1103/PhysRevD.76.013009
report-no: ANL-HEP-PR-07-12
categories: hep-ph
license: None
abstract:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
more detailed tests w

In [93]:
# Create a dictionary mapping shortened names of months to numerical values
month_mapping = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
}

In [94]:
# #count lines in file
# num_lines = sum(1 for line in open(data_path))
# print(num_lines)

In [95]:
def get_v1_submission_date(paper_info):
    # Access the first version of the paper
    v1_created_date = paper_info['versions'][0]['created']
    
    # Extracting only the month and year
    v1_month_year = v1_created_date.split(',')[1].strip()

    # Extract month and year separately
    month, year = month_mapping[v1_month_year.split()[1]], int(v1_month_year.split()[2])
    
    return [month, year]

# Function to save subset metadata to the file
def append_to_file(subset_metadata, json_file_path):
    with open(json_file_path, 'a') as json_file:
        for e in subset_metadata:
            json.dump(e, json_file)
            json_file.write('\n')  # Add newline for readability

In [98]:
def clear_file(file_path):
    with open(file_path, 'w') as f:
        pass

In [99]:
def create_subset(metadata, subset_metadata_path, max_papers=1000):
    # Subset of the metadata with publish date extracted
    subset_metadata = []

    with open(subset_metadata_path, 'w') as json_file:
        pass

    # Iterate through each paper in the metadata
    num_processed = 0

    for i, paper_info in enumerate(metadata):

        # Skip papers that are not in the CS category
        categories = paper_info.get('categories')
        if not (' cs.' in categories or categories.startswith('cs.')):
            continue

        paper_dict = {
            'id': paper_info.get('id'),
            'title': paper_info.get('title'),
            'abstract': paper_info.get('abstract'),
            'categories': paper_info.get('categories'),
            'submit_date': get_v1_submission_date(paper_info)
        }

        subset_metadata.append(paper_dict)

        if num_processed % 100 == 0:
            append_to_file(subset_metadata, subset_metadata_path)
            print(f"Processed {num_processed} papers")
            subset_metadata = []

        num_processed += 1

        if num_processed > max_papers:
            break

clear_file(subset_metadata_path)
create_subset(metadata, subset_metadata_path, max_papers=20000)

Processed 0 papers
Processed 100 papers
Processed 200 papers
Processed 300 papers
Processed 400 papers
Processed 500 papers
Processed 600 papers
Processed 700 papers
Processed 800 papers
Processed 900 papers
Processed 1000 papers
Processed 1100 papers
Processed 1200 papers
Processed 1300 papers
Processed 1400 papers
Processed 1500 papers
Processed 1600 papers
Processed 1700 papers
Processed 1800 papers
Processed 1900 papers
Processed 2000 papers
Processed 2100 papers
Processed 2200 papers
Processed 2300 papers
Processed 2400 papers
Processed 2500 papers
Processed 2600 papers
Processed 2700 papers
Processed 2800 papers
Processed 2900 papers
Processed 3000 papers
Processed 3100 papers
Processed 3200 papers
Processed 3300 papers
Processed 3400 papers
Processed 3500 papers
Processed 3600 papers
Processed 3700 papers
Processed 3800 papers
Processed 3900 papers
Processed 4000 papers
Processed 4100 papers
Processed 4200 papers
Processed 4300 papers
Processed 4400 papers
Processed 4500 papers


In [13]:
# Open the file and read its contents
num_first_lines = 10
with open(subset_metadata_path, 'r') as json_file:
    # Read the first 10 lines
    first_10_lines = [json_file.readline().strip() for _ in range(num_first_lines)]

# Print the first 10 lines of the file
for line in first_10_lines:
    print(line)


{"id": "0712.1359", "title": "Borel Ranks and Wadge Degrees of Context Free Omega Languages", "abstract": "  We show that, from a topological point of view, considering the Borel and the\nWadge hierarchies, 1-counter B\\\"uchi automata have the same accepting power\nthan Turing machines equipped with a B\\\"uchi acceptance condition. In\nparticular, for every non null recursive ordinal alpha, there exist some\nSigma^0_alpha-complete and some Pi^0_alpha-complete omega context free\nlanguages accepted by 1-counter B\\\"uchi automata, and the supremum of the set\nof Borel ranks of context free omega languages is the ordinal gamma^1_2 which\nis strictly greater than the first non recursive ordinal. This very surprising\nresult gives answers to questions of H. Lescow and W. Thomas [Logical\nSpecifications of Infinite Computations, In:\"A Decade of Concurrency\", LNCS\n803, Springer, 1994, p. 583-621].\n", "categories": "cs.LO cs.GT math.LO", "submit_date": [12, 2007]}
{"id": "0712.1363", "t

In [14]:
subset_metadata_path_without_concepts = "/mnt/qnap/liranc6/data/con_graph/dataset/subset_metadata.json"
subset_metadata_path_with_concepts = "/mnt/qnap/liranc6/data/con_graph/dataset/subset_metadata_with_concepts.json"
corpus_path = "/home/liranc6/con_graph/corpus.txt"

In [15]:
a = set()
metadata_with_concepts = get_metadata(subset_metadata_path_with_concepts)
for i, paper in enumerate(metadata_with_concepts):
    for c in paper['concepts']:
        a.update(c)
        # print()

    # print("\n")
    if i > 100:
        break

print(F"{len(a)=}")
        

len(a)=66


In [ ]:
tmp_path = "/mnt/qnap/liranc6/data/con_graph/dataset/tmp.json"

def str_list_to_lower(lst):
    return [x.lower() for x in lst]

corpus = set()
with open(tmp_path, 'w') as g:
    for i, paper in enumerate(get_metadata(subset_metadata_path_with_concepts)):
        # paper['concepts'] = str_list_to_lower(paper['concepts'])
        corpus.update(paper['concepts'])
        # json.dump(paper, g)

print(f"{len(corpus)=}")


In [80]:
corpus = c2

In [81]:
with open(corpus_path, 'w') as corpus_file:
            print(f"{len(corpus)=}")
            corpus = '\n'.join(corpus)    
            corpus_file.write(corpus + '\n')

len(corpus)=7317


In [103]:
for p in get_metadata(subset_metadata_path_with_concepts):
    print(p)
    break

{'id': '0906.4492', 'title': 'Efficient Generation of Craig Interpolants in Satisfiability Modulo\n  Theories', 'abstract': '  The problem of computing Craig Interpolants has recently received a lot of\ninterest. In this paper, we address the problem of efficient generation of\ninterpolants for some important fragments of first order logic, which are\namenable for effective decision procedures, called Satisfiability Modulo Theory\nsolvers.\n  We make the following contributions.\n  First, we provide interpolation procedures for several basic theories of\ninterest: the theories of linear arithmetic over the rationals, difference\nlogic over rationals and integers, and UTVPI over rationals and integers.\n  Second, we define a novel approach to interpolate combinations of theories,\nthat applies to the Delayed Theory Combination approach.\n  Efficiency is ensured by the fact that the proposed interpolation algorithms\nextend state of the art algorithms for Satisfiability Modulo Theories. 

In [102]:
c2 = set()
for i, p in enumerate(get_metadata(subset_metadata_path_with_concepts)):
    c2.update(p['concepts'])
    # print(i)

print(i)
print(f"{len(c2)=}")

43
len(c2)=205


In [83]:
with open(corpus_path, 'r') as f:
    corpus = f.read().split('\n')

print(corpus[:1000])


['microelectrode design', 'information integration', 'feasibility tests', 'distributed systems', 'bandwidth allocation', 'architecture design', 'routing games', 'automated theorem proving', 'edge-based routing', 'hasse diagrams visualization', 'semantic modeling', 'streaming data analysis', 'entanglement representation', 'transaction management', 'probabilistic argumentation systems', 'maximum-weight balanced cut', 'loss-leader pricing', 'computational complexity', 'package fabrication', 'etching techniques', 'vertex-wise budgeting', 'password authentication', 'electrical properties', 'computational neuroscience', 'stokes flow solvers', 'instantaneous power constraint', 'game theory', 'spreadsheet error-checking', 'computational overhead', 'shannon capacity', 'channel coding theory', 'morphing algorithm', 'kronecker product algorithm', 'minimum mean-square error', 'half-duplex operation', 'system reliability', 'edge congestion', 'vertical partitioning', 'scaling window estimation', 'po

In [85]:
save_subset = "/mnt/qnap/liranc6/data/con_graph/dataset/save_subset.json"
save_corpus = "/mnt/qnap/liranc6/data/con_graph/dataset/save_corpus.txt"

!cp {subset_metadata_path_with_concepts} {save_subset}
!cp {corpus_path} {save_corpus}
